## Collecting and Saving Images as LMDB files

Note: KC = 0, Peace = 1

In [3]:
import re
import cv2
import lmdb
import pickle
import os
from tqdm import tqdm
from sklearn import preprocessing
import numpy as np
import matplotlib.pyplot as plt

In [49]:
def capture_faces(num_faces=1000, face_size=(128, 128)):
    
    name = str(input("Please enter your first name: "))
    output_path = os.path.join("data", f"{name}.lmdb")
    # Initialize face detector
    facedetect = cv2.CascadeClassifier('models\haarcascade_frontalface_default.xml')
    
    # Create LMDB environment
    map_size = num_faces * 1024 * 1024 * 3  
    env = lmdb.open(output_path, map_size=map_size)

    # Initialize webcam
    cam = cv2.VideoCapture(0)
    
    saved_count = 0
    progress = tqdm(total=num_faces, desc="Saving faces")

    try:
        with env.begin(write=True) as txn:
            while saved_count < num_faces:
                ret, frame = cam.read()
                if not ret:
                    continue

                # Detect faces
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                faces = facedetect.detectMultiScale(gray, 1.3, 5)

                for (x, y, w, h) in faces:
                    # Extract and resize face
                    face_img = frame[y:y+h, x:x+w]
                    resized_face = cv2.resize(face_img, face_size)

                    # Store in LMDB
                    key = f"face_{saved_count:08d}".encode()
                    txn.put(key, pickle.dumps(resized_face))
                    
                    saved_count += 1
                    progress.update(1)
                    
                    # Display current face count on the frame
                    cv2.putText(frame, str(saved_count), (50,50), cv2.FONT_HERSHEY_COMPLEX, 1, (50,50,255), 1)
        
                    # Draw rectangle around detected face
                    cv2.rectangle(frame, (x,y), (x+w, y+h), (50,50,255), 1)
              
                # Show preview
                cv2.imshow('Face Capture', frame)
                
                # Exit on key ('q') pressed or completion (100 faces collected)
                k=cv2.waitKey(1)
                if k==ord('q') or len(faces_data)==1000:
                    break

    finally:
        cam.release()
        cv2.destroyAllWindows()
        progress.close()
        print(f"\nSaved {saved_count} faces")



# Capture 1000 faces to database
capture_faces()
   


<>:12: SyntaxWarning: invalid escape sequence '\h'
<>:12: SyntaxWarning: invalid escape sequence '\h'
C:\Users\DELL\AppData\Local\Temp\ipykernel_15484\3023620433.py:12: SyntaxWarning: invalid escape sequence '\h'
  facedetect = cv2.CascadeClassifier('models\haarcascade_frontalface_default.xml')


Please enter your first name:  KC


Saving faces: 100%|████████████████████████████████████████████████████████████████| 1000/1000 [01:05<00:00, 15.22it/s]


Saved 1000 faces


In [59]:
def view_image(lmdb_path, key_to_view):

    # Open LMDB environment in read-only mode
    env = lmdb.open(lmdb_path, readonly=True)
    
    with env.begin() as txn:
        # Retrieve the serialized image data using the key
        value = txn.get(key_to_view.encode())
        if value is None:
            print(f"No data found for key: {key_to_view}")
            return
        
        # Deserialize the data back into a numpy array
        image = pickle.loads(value)
        
        # Display the image using OpenCV
        cv2.imshow(f"Image: {key_to_view}", image)
        cv2.waitKey(0)  # Wait for a key press to close the window
        cv2.destroyAllWindows()


lmdb_path = "data\Peace.lmdb" 
key_to_view = "face_00000001"  
view_image(lmdb_path, key_to_view)


<>:22: SyntaxWarning: invalid escape sequence '\P'
<>:22: SyntaxWarning: invalid escape sequence '\P'
C:\Users\DELL\AppData\Local\Temp\ipykernel_15484\2113577972.py:22: SyntaxWarning: invalid escape sequence '\P'
  lmdb_path = "data\Peace.lmdb"


## Load Training Data

In [4]:
def get_images(lmdb_path, features, labels):
   
    env = lmdb.open(lmdb_path, readonly=True)
    
    pattern = r"\\(.*)\."
    match = re.search(pattern, lmdb_path)
    name = match.group(1)[:]
    
    
    with env.begin() as txn:
        cursor = txn.cursor()
        for key, value in cursor:
            key_str = key.decode()  # Decode key from bytes to string
            key_str += name
            image = pickle.loads(value)  # Deserialize image data back into a numpy array
            labels.append(name)
            features.append(image)
    
    return features, labels




In [5]:
dir = "data"
features = []
labels = []
for path in os.listdir(dir):
    lmdb_path = os.path.join(dir,path)
    features, labels = get_images(lmdb_path, features, labels)

In [9]:
label_encoder = preprocessing.LabelEncoder()
labels = label_encoder.fit_transform(labels)

In [16]:
len(labels)

2000

In [11]:
features = np.array(features) 
labels = np.array(labels)

In [20]:
np.unique(labels)

array([0, 1], dtype=int64)

## Train Model 

In [13]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Input,
    Conv2D, 
    Activation, 
    BatchNormalization, 
    MaxPooling2D, 
    Dropout, 
    Flatten, 
    Dense
)
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler,EarlyStopping,TensorBoard,ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

In [14]:
X_train, X_temp, y_train, y_temp = train_test_split(features, labels, test_size=0.3, random_state=4)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=4)

In [81]:
# Initiating model on CPU
def build():
    model = Sequential()
    model.add(Input(shape = (128, 128, 3)))
    # Feature Learning Layers with Kernel size (3x3), Step size (1 pixel)
    model.add(Conv2D(32,(3, 3),strides=(1, 1),padding='same'))
    model.add(Activation('relu'))# Activation function
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2,2), padding = 'same'))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (5,5), padding = 'same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2,2), padding = 'same'))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, (3,3), padding = 'same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2,2), padding = 'same'))
    model.add(Dropout(0.3))

    model.add(Conv2D(256, (5,5), padding = 'same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2,2), padding = 'same'))
    model.add(Dropout(0.3))

    model.add(Conv2D(512, (3,3), padding = 'same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2,2), padding = 'same'))
    model.add(Dropout(0.3))

    # Flattening tensors
    model.add(Flatten())

    # Fully-Connected Layers
    model.add(Dense(2048))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    # Output Layer
    model.add(Dense(1, activation = 'sigmoid')) # binaryClassification layer
    return model

In [83]:
model = build()

In [85]:
# Compiling model
model.compile(optimizer = 'adam',
              loss = 'binary_crossentropy', # change to sparse_categorical_crossentropy for multiclass where each sample can only belong to one class
              metrics = ['accuracy']) # Evaluation metric

In [87]:
# Defining an Early Stopping and Model Checkpoints
early_stopping = EarlyStopping(monitor = 'val_loss', 
                               min_delta = 0.01,
                               mode = 'min',
                               patience = 7,
                               start_from_epoch = 20,
                              restore_best_weights = True)

checkpoint = ModelCheckpoint('best_model.keras',
                            monitor = 'val_loss',
                            save_best_only = True)

In [89]:
model.fit(X_train, y_train, epochs=100, batch_size=64, verbose=  1, validation_data = (X_val, y_val), callbacks=[checkpoint, early_stopping])

Epoch 1/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 46s 2s/step - accuracy: 0.8439 - loss: 2.8288 - val_accuracy: 0.5100 - val_loss: 3468.0435
Epoch 2/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 46s 2s/step - accuracy: 0.9929 - loss: 0.1702 - val_accuracy: 0.5100 - val_loss: 3473.4360
Epoch 3/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 55s 3s/step - accuracy: 0.9990 - loss: 0.0125 - val_accuracy: 0.5100 - val_loss: 932.8412
Epoch 4/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 57s 3s/step - accuracy: 0.9992 - loss: 0.0396 - val_accuracy: 0.5167 - val_loss: 277.9084
Epoch 5/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 58s 3s/step - accuracy: 0.9997 - loss: 0.0015 - val_accuracy: 0.7100 - val_loss: 55.5258
Epoch 6/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 56s 3s/step - accuracy: 0.9993 - loss: 0.0061 - val_accuracy: 0.8633 - val_loss: 10.6264
Epoch 7/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 334s 16s/step - accuracy: 1.0000 - loss: 2.9549e-08 - val_accuracy: 0.9600 - val_loss: 3.4463
Epoch 8/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 63s 3s/step - accuracy: 0.9991 - loss: 0.0168 - val_ac

In [15]:
# Load the best model
best_model = tf.keras.models.load_model('best_model.keras')

## Evaluate Model

In [34]:
best_model.evaluate(X_test, y_test, batch_size=64, verbose=1) 

5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 372ms/step - accuracy: 0.9989 - loss: 0.0056  


[0.016857018694281578, 0.996666669845581]

## Save Model

In [95]:
with open('face_recog_model.pkl', 'wb') as file:  # 'wb' for writing in binary mode
    pickle.dump(best_model, file)

## Test Case

Get files of pictures and labels for test predictions and evaluate.

### Collect Face from Camera

In [42]:
def scan_face(num_faces=1, face_size=(128, 128)):
    
    face_data = []
    # Initialize face detector
    facedetect = cv2.CascadeClassifier('models\haarcascade_frontalface_default.xml')
    
    # Initialize webcam
    cam = cv2.VideoCapture(0)
    
    while True:
      # Read a frame from the video source (webcam/file)
      ret,frame =  cam.read()
      if not ret:
        print("Error reading frame from camera")
        continue
        
      # Convert frame to grayscale (face detection works better on grayscale)
      gray=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
      # Detect faces
      faces = facedetect.detectMultiScale(gray, 1.3, 5)
      
      for (x, y, w, h) in faces:
        # Draw rectangle around detected face
        cv2.rectangle(frame, (x,y), (x+w, y+h), (50,50,255), 1) 
        # Extract and resize face
        cropped_face = frame[y:y+h, x:x+w]
        resized_face = cv2.resize(cropped_face , face_size)
        face_data.append(np.array(resized_face))
        # Display feedback
        cv2.putText(frame, f"Capturing ", 
                   (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        # Show preview
      cv2.imshow('Face Capture', frame)
      if len(face_data) != 1:
          cv2.waitKey(0)
          print("No face found")
          break
      else: 
          cv2.waitKey(0)
          print(f"\nFace Scanned")
          break

    cam.release()
    cv2.destroyAllWindows()
        
    return face_data


<>:5: SyntaxWarning: invalid escape sequence '\h'
<>:5: SyntaxWarning: invalid escape sequence '\h'
C:\Users\DELL\AppData\Local\Temp\ipykernel_12520\2663604764.py:5: SyntaxWarning: invalid escape sequence '\h'
  facedetect = cv2.CascadeClassifier('models\haarcascade_frontalface_default.xml')


In [48]:
face = scan_face()


Face Scanned


In [ ]:
face_dim = np.expand_dims(face[0], axis=0)

In [52]:
face_dim.shape

(1, 128, 128, 3)

In [54]:
best_model.predict(face_dim).astype(int)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step


array([[1]])

Correctly predicted the identity of the face. 1 is for Peace (that's me!)

In [18]:
def predict(face_dim):
    prediction = best_model.predict(face_dim).astype(int)
    if prediction == 0:
        print("Access granted. Welcome KC!")
    elif prediction == 1:
        print("Access granted. Welcome Peace!")
    else:
        print("Access denied!!!")

In [73]:
predict(face_dim)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
Access granted. Welcome Peace!


### Collect Image from Directory

Let's try an image from my gallery of KC.

In [29]:
img_input = "pic1.jpg"
img = cv2.imread(img_input)
cv2.imshow("image", img)
cv2.waitKey(0) 
cv2.destroyAllWindows()


## Preprocess Image

In [31]:
def preprocess(img, face_size=(128, 128)):
    img_array = []
    
    # Initialize face detector
    facedetect = cv2.CascadeClassifier('models\haarcascade_frontalface_default.xml')
    
    # Detect faces
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    face = facedetect.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in face:
        # Extract and resize face
        face_img = img[y:y+h, x:x+w]
        resized_face = cv2.resize(face_img, face_size)
        img_array.append(np.array(resized_face))
    return img_array
         


<>:5: SyntaxWarning: invalid escape sequence '\h'
<>:5: SyntaxWarning: invalid escape sequence '\h'
C:\Users\DELL\AppData\Local\Temp\ipykernel_5212\1381088494.py:5: SyntaxWarning: invalid escape sequence '\h'
  facedetect = cv2.CascadeClassifier('models\haarcascade_frontalface_default.xml')


In [33]:
face1 = preprocess(img)

In [35]:
face1_dim = np.expand_dims(face1[0], axis=0)

In [37]:
predict(face1_dim)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 521ms/step
Access granted. Welcome KC!


It works perfectly! Hurray!!!